<a href="https://colab.research.google.com/github/sporella/datos_geoespaciales_python/blob/main/datos_espaciales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalar librerías necesarias

En Google Colab ya se encuentran instaladas algunas librerías por defecto, pero para trabajar con datos geoespaciales debemos instalar algunas más.

Podemos utilizar el instalador de paquetes `pip` al igual como se haría en la terminal de tu propio computador. 

> Para utilizar códigos que normalmente escribiríamos en la terminal, debemos anteponer `!`

Instalaremos:

- `pygeos` herramientas para análisis geoespaciales
- `folium` para visualizaciones dinámicas (incluir `-U` para instalar la última versión)
- `geopandas` para trabajar con datos vectoriales
- `mapclassify` contiene clasificaciones para mapas de cloropletas

In [ ]:
! pip install pygeos
! pip install folium -U
! pip install geopandas
! pip install mapclassify


# Introducción
## Primero debemos importar las librerías necesarias
En python podemos importar las librerías con un alias, para geopandas se utiliza normalmente `gpd`.

`geopandas`nos permite trabajar con datos vectoriales (puntos, líneas, polígonos, etc) en un formato tabular que nos permite realizar procesamiento de datos con la librería `pandas`.

In [ ]:
import geopandas as gpd
import pandas as pd
import folium

## Leer datos

En este primer ejemplo trabajaremos con datos que trae geopandas. Son datos de Natural Earth con información de países (polígonos) y ciudades capitales (puntos).


In [ ]:
paises = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
paises.head()

In [ ]:
capitales = gpd.read_file(gpd.datasets.get_path("naturalearth_cities"))
capitales.head()

## Gráfico estático

Podemos hacer un gráfico estático con el método plot() que utiliza la sintaxis de `matplotlib`

In [ ]:
paises.plot();
paises.plot("continent");
paises.plot("pop_est");

In [ ]:
capitales.plot();

## Gráfico dinámico

En sus versiones más recientes, geopandas nos permite usar la librería `folium`para hacer visualizaciones dinámicas mediante el método `.explore()`

In [ ]:
paises.explore()

In [ ]:
capitales.explore()

## Proyección espacial

Como vimos antes, es muy importante conocer la proyección en la que se encuentran nuestros datos, o incluso si es que tienen o no una proyección.

Trabajar con datos en proyecciones diferentes nos lleva a incurrir en errores que no siempre son fáciles de detectar, por lo que siempre es necesario chequear las proyecciones.


### Proyección actual

Podemos consultar la proyección de un set de datos utilizando `.crs`

In [ ]:
paises.crs

### Cambiar proyección

Para cambiar la proyección, utilizamos el método `to_crs()` e incluimos el número del epsg de destino.

Podemos consultar diferentes epsg en https://epsg.io/

In [ ]:
paises_psm = paises.to_crs(3857)
paises_psm.crs

In [ ]:
paises.head()

In [ ]:
paises_psm.head()

In [ ]:
paises.plot(figsize = [10, 10]);

In [ ]:
paises_psm.plot(figsize = [10, 10]);

In [ ]:
paises_psm.explore()

Existen proyecciones específicas para ciertos lugares del mundo, como por ejemplo, la Antártica

In [ ]:
antartica = paises[paises.continent == "Antarctica"]
antartica.plot()

In [ ]:
antartica_polar = antartica.to_crs(3031) # 5481
antartica_polar.plot()


In [ ]:
antartica_polar = antartica.to_crs(5481) # 5481
antartica_polar.plot()


# Trabajando con nuestros propios datos

Por lo general, tendremos tres fuentes principales desde las cuales podemos obtener datos geospaciales:

- **Archivos csv con coordenadas** de ubicaciones de algún tópico de interés

- **Capas de servicios públicos o que estén en internet**. Son capas de polígonos, puntos o líneas que alguien más ha disponibilizado y que por lo general se encuentran en formato Shapefile (GIS), KML (GoogleEarth) o geojson.

- **Capas hechas por nosotros mismos** es la forma menos común de obtener los datos, ya que debemos tener experiencia en el uso de Sistemas de Información Geográfica.

## Algunas fuentes de datos en Internet

- [geonode CEDEUS](http://datos.cedeus.cl/)

- [IDE Chile](https://www.ide.cl/)

- [geodatos INE](https://www.ine.cl/herramientas/portal-de-mapas/geodatos-abiertos)

- [Observatorio Ciudades UC](https://www.ine.cl/herramientas/portal-de-mapas/geodatos-abiertos)

## Cargar archivos

Geopandas nos permite cargar archivos geoespaciales de diferentes tipos, como .`shp.`, `.geojson`, `csv`, etc.


### Archivo shp

Para cargar un archivo shp debemos poner la ruta del archivo shp., en el mismo directorio deben estar los otros archivos que componen el shapefile(.cpg, .dbf, .prj, .shx)

In [ ]:
comunas_valparaiso = gpd.read_file("comunas_valparaiso.shp")
comunas_valparaiso.plot(figsize = [8,8])
comunas_valparaiso.explore()

### Archivo Geojson

Podemos leerlo desde el almacenamiento local o desde una url.

In [ ]:
transporte = gpd.read_file("http://datos.cedeus.cl/geoserver/wfs?srsName=EPSG%3A4326&typename=geonode%3Atransporte_valpo&outputFormat=json&version=1.0.0&service=WFS&request=GetFeature")
#transporte["route_desc"] = transporte.route_desc.replace("", "Sin Información")
transporte.explore()
transporte.head()

In [ ]:
transporte.route_desc.unique()

In [ ]:
transporte["route_desc"] = transporte.route_desc.replace("", "Sin información")
transporte

In [ ]:
transporte.explore("route_desc")

### Archivos csv

Cargamos primero con pandas `pd.read_csv()` y luego convertimos con `gpd.GeoDataFrame()`

In [ ]:
monumentos = pd.read_csv("monumentos.csv")
monumentos.head()

In [ ]:
type(monumentos)

In [ ]:
monumentos = gpd.GeoDataFrame(monumentos, geometry = gpd.points_from_xy(monumentos.lon, monumentos.lat))
monumentos.explore()

En este caso es importante chequear la información de proyección geoespacial con `.crs` para darnos cuenta que debemos asignar el sistema de coordenadas al realizar la conversión:

In [ ]:
print(monumentos.crs)

In [ ]:
monumentos = gpd.GeoDataFrame(monumentos, geometry = gpd.points_from_xy(monumentos.lon, monumentos.lat), crs = 4326) # inlcuir el argumento crs
monumentos.explore()

## Cruce de información

Nos gustaría generar un mapa solo para la región de Valparaíso pero nuestros puntos de monumentos nacionales no poseen información de región, comuna y provincia.

En este caso, podemos utilizar un "join espacial". Para esto, lo primero que debemos hacer es confirmar que ambas capas tengan el mismo sistema de proyección:

In [ ]:
comunas_valparaiso.crs == monumentos.crs

In [ ]:
comunas_valparaiso.crs

In [ ]:
monumentos.crs

Entonces tenemos que convertir las capas al mismo sistema de coordenadas. Utilizaremos una proyección en metros (la de las comunas), así que transformaremos los monumentos a psm (3857) 

In [ ]:
monumentos_psm = monumentos.to_crs(comunas_valparaiso.crs)
comunas_valparaiso.crs == monumentos_psm.crs

In [ ]:
monumentos.sjoin(comunas_valparaiso).shape

Y ahora podremos hacer el join

In [ ]:
monumentos_psm.shape
comunas_valparaiso.shape

In [ ]:
monumentos_psm.sjoin(comunas_valparaiso).shape

In [ ]:
monumentos_valparaiso = monumentos_psm.sjoin(comunas_valparaiso[["Region",	"Comuna",	"Provincia", "geometry"]])
monumentos_valparaiso.explore()

In [ ]:
monumentos_valparaiso.explore("Comuna", tiles = "cartoDBPositron")

In [ ]:
transporte.crs
transporte_psm = transporte.to_crs(3857)

In [ ]:

monumentos_valparaiso["distancia_km"] = monumentos_valparaiso.geometry.apply(lambda x: transporte_psm.distance(x).min()/1000)
monumentos_valparaiso.explore("distancia_km", cmap = "YlGnBu_r")

# Mejorando nuestra visualización

En esta sección trabajaremos de forma más libre, poniéndo énfasis en la importancia de consultar la documentación para obtener resultados más personalizados.

In [ ]:
m = comunas_valparaiso.explore(column = "Provincia", 
                               cmap = ["#00e3d4","#9f0f08","#ac9bff","#ecce42","#4a308c","#007735","#ff97bf"],
                               tooltip = False,
                               popup = ["Comuna", "Provincia"])
monumentos_valparaiso.explore(m = m, marker_type = "circle_marker", marker_kwds = {"color" : "#f5e9f7"})
transporte_psm.explore(m = m, column = "route_desc")
folium.LayerControl().add_to(m)
m

In [ ]:
comunas_valparaiso.Provincia.nunique()